In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


In [2]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'ExecuteTime': {
   	'display_absolute_timestamps': False,
    'relative_timing_update_period': 5,
    'template': {
    	'executed': 'started ${start_time}, finished in ${duration}',
    }
}})

{'Cell': {'cm_config': {'lineNumbers': True}},
 'ExecuteTime': {'display_absolute_timestamps': False,
  'relative_timing_update_period': 5,
  'template': {'executed': 'started ${start_time}, finished in ${duration}'}}}

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [3]:
sample_num = 100
enr = np.random.uniform(low=3, high=5, size=(sample_num,))
br = np.random.uniform(low=30000, high=50000, size=(sample_num,))
enr_br = np.column_stack((enr,br))
print(np.shape(enr_br))
enr_br

(100, 2)


array([[  4.41020027e+00,   4.25483886e+04],
       [  4.80515853e+00,   3.27288336e+04],
       [  4.95753263e+00,   4.59086682e+04],
       [  3.49672915e+00,   4.99217955e+04],
       [  4.75260688e+00,   3.66612922e+04],
       [  4.92103353e+00,   4.74000033e+04],
       [  4.01696519e+00,   3.42126046e+04],
       [  4.29548078e+00,   3.83611886e+04],
       [  4.99844791e+00,   4.29986837e+04],
       [  3.51643762e+00,   3.35897757e+04],
       [  4.89189837e+00,   4.34754959e+04],
       [  4.69677881e+00,   3.17992379e+04],
       [  4.49771772e+00,   4.21714799e+04],
       [  4.12347666e+00,   4.72277157e+04],
       [  3.86497140e+00,   4.86684467e+04],
       [  4.26894259e+00,   3.08149895e+04],
       [  3.33008537e+00,   4.55188710e+04],
       [  4.72827331e+00,   3.58747328e+04],
       [  4.14725458e+00,   3.22194817e+04],
       [  4.25240712e+00,   3.71238832e+04],
       [  4.73808267e+00,   4.63246507e+04],
       [  3.08651626e+00,   4.15125262e+04],
       [  

## Pickle and model

In [4]:
# load pickled dictionary with
# key: isotope
# value: trained model
model_dict = pickle.load(open('./script/ann_model.pkl',"rb"))

In [16]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition

def predict(enr_bu):
    model = model_dict['model']
    iso_list = model_dict['iso_list']
    comp_dict = {}
    for indx, i in enumerate(enr_br):
        x = model_dict['xscaler'].transform([i])
        y = model_dict['yscaler'].inverse_transform(model.predict(x))
        y = y[0]
        comp_dict[indx] = {}
        for indx2, iso in enumerate(iso_list):
            comp_dict[indx][iso] = y[indx2]
    return comp_dict


%time predict(enr_br)


CPU times: user 207 ms, sys: 205 ms, total: 412 ms
Wall time: 155 ms


{0: {'zr-95': 0.00013412905,
  'zr-93': 0.098992832,
  'ra-226': 7.0858245e-12,
  'ru-103': 7.7518871e-06,
  'ru-106': 0.0097117182,
  'sb-124': 1.0602422e-07,
  'sb-125': 0.0010887302,
  'pu-238': 0.031060519,
  'pu-239': 0.72713119,
  'np-237': 0.065170519,
  'nb-95': 0.00015820807,
  'pa-231': 6.709454e-08,
  'pd-107': 0.030832885,
  'pm-147': 0.015320909,
  'pu-240': 0.27683243,
  'pu-241': 0.18290022,
  'pu-244': 6.3841471e-06,
  'pu-242': 0.070804097,
  'i-129': 0.020599952,
  'h-3': 6.7476299e-06,
  'kr-85': 0.0031808743,
  'nb-94': 2.0230277e-07,
  'cm-248': 8.7288782e-08,
  'cs-134': 0.013120483,
  'cs-135': 0.059657834,
  'cm-245': 0.00065983622,
  'cm-244': 0.0083664358,
  'cm-246': 6.394786e-05,
  'cm-247': 1.0682636e-06,
  'eu-152': 1.9893014e-06,
  'cs-137': 0.15036967,
  'eu-154': 0.0036276681,
  'eu-155': 0.00098705234,
  'am-241': 0.016984703,
  'am-242m': 0.00013171919,
  'am-243': 0.019962162,
  'cf-251': 2.3534841e-10,
  'cf-249': 8.3395751e-10,
  'ag-108m': 2.94781

## Looking up from db

In [6]:

df = pd.read_csv('./curated.csv', index_col=0)

In [17]:
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
def deplete_lookup(df, enr_br):
    depleted_composition_lookup = {}
    for indx, enr_br_set in enumerate(enr_br):
        df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
        minimum_err = df['err'].argmin()
        df = df.drop(columns='err')
        average_assem = df.loc[minimum_err]
        depleted_composition_lookup[indx] = average_assem.iloc[5:].to_dict()
    return depleted_composition_lookup

%time deplete_lookup(df, enr_br)

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys


CPU times: user 7.4 s, sys: 14 s, total: 21.4 s
Wall time: 21.4 s


{0: {'zr-95': 0.00017216386278195487,
  'zr-93': 0.10024109492481202,
  'ra-226': 4.1085829417293227e-12,
  'ru-103': 9.2872781954887209e-06,
  'ru-106': 0.010664275610902257,
  'sb-124': 1.0842815554511278e-07,
  'sb-125': 0.0011478955357142858,
  'pu-238': 0.02632386278195489,
  'pu-239': 0.70842840695488729,
  'np-237': 0.066018451597744346,
  'nb-95': 0.00020310254229323303,
  'pa-231': 5.3134527725563913e-08,
  'pd-107': 0.027726132518796992,
  'pm-147': 0.017719288768796998,
  'pu-240': 0.25394358552631568,
  'pu-241': 0.17732997885338345,
  'pu-244': 5.1411994830827076e-06,
  'pu-242': 0.062418914473684216,
  'i-129': 0.019905380404135337,
  'h-3': 6.6038977913533827e-06,
  'kr-85': 0.0037003963815789467,
  'nb-94': 2.066818280075188e-07,
  'cm-248': 3.6221123120300754e-08,
  'cs-134': 0.013483809680451129,
  'cs-135': 0.053023033364661652,
  'cm-245': 0.00043433916823308258,
  'cm-244': 0.0060603942669172932,
  'cm-246': 3.6172805451127824e-05,
  'cm-247': 5.1933905075187974e-0